## Get information and crop geo_tif files

In [1]:
from osgeo import gdal, osr 
import numpy as np    
import os
import cv2

#### Defining a class of utils_tif and methos to returns the desired information and crop the tiff image

In [2]:
class Utils_tif:
    """
    Class that returns a desired information and crop of a tif file  
    """
        
    def get_information_from_tif(self, path_to_tif):
        """
        Function that returns a coordinates system, pixel resolution, size (height, width) and latitude/longitude 
        from a tif file

        This Function receives:
            path_to_tif: path to tif file to get the informations

        return: a dictionary containing the name of information and your value. 
        return example: 
            {'coordinate_system': 'SIRGAS 2000', # str
             'pixel_resolution': [0.1, 0.1], # list
             'size': [2396, 3247], # list
             'lat_long': (-19.06212225267312, -49.70194997381457, 0.0) # tuple
             }
        """

        # setup
        
        information_dict = {}
        img = gdal.Open(img_path)
        projetion = img.GetProjection()

        # get the coordinates system
        srs=osr.SpatialReference(wkt=projetion) 
        coordinate_system = srs.GetAttrValue('geogcs')

        # get the pixel resolution
        gt = img.GetGeoTransform()
        pixel_X_resolution = gt[1]
        pixel_Y_resolution = gt[5]

        # get the length images in pixels
        width = img.RasterXSize
        height = img.RasterYSize

        #get lat long from a tif
        srs.ImportFromWkt(img.GetProjectionRef())

        # create the new coordinate system
        new_cs = osr.SpatialReference()
        new_cs.ImportFromEPSG(4326)

        # create a transform object to convert between coordinate systems
        transform = osr.CoordinateTransformation(srs,new_cs) 

        minx = gt[0]
        miny = gt[3] + width*gt[4] + height*gt[5] 

        #get the coordinates in lat long
        lat_long = transform.TransformPoint(minx,miny) 

        information_dict['coordinate_system'] = coordinate_system
        information_dict['pixel_resolution']  = [pixel_X_resolution, -pixel_Y_resolution]
        information_dict['size']              = [width, height]
        information_dict['lat_long']          = lat_long

        return information_dict

    def crop_image_and_save_windows(self, path_to_tif, len_window, path_to_save):
        
        """
        Function that crop a tif file in integers of size len_window x len_window and save then in path_to_save

        This Function receives:
            path_to_tif : path to tif file
            len_window  : length of croped window to crop 
            path_to_save: path to save all the croped files of tif file

        return: Nothing
        """
        
        img = gdal.Open(path_to_tif)
        width = img.RasterXSize
        height = img.RasterYSize
        
        # calculating the all windows in that tif
        times_in_X = int(width/len_window)
        times_in_Y = int(height/len_window)
        
        numpy_img = img.ReadAsArray()
        
        # transforming the shape of chanels first to chanels last (3,width, height) --> (width,heigth,3)
        numpy_img = np.swapaxes(numpy_img,0,2)
        
        """
        Note this approach of crop looses the information of tif file.
        
        But as the choice to keep this information depends on the problem, 
        in this case I split the image without keeping it
        """
        for x in range(times_in_X):
            for y in range(times_in_Y):
                crop_img = numpy_img[x*len_window:(x+1)*len_window, y*len_window:(y+1)*len_window]
                cv2.imwrite(f'{path_to_save}/{x}_{y}.png', crop_img)

#### Get the information of a first tif file

In [3]:
utils_tif = Utils_tif()

img_path = 'talhoes_tifs/4039_01_04 (1).tif'
utils_tif.get_information_from_tif(img_path)

{'coordinate_system': 'SIRGAS 2000',
 'pixel_resolution': [0.1, 0.1],
 'size': [2396, 3247],
 'lat_long': (-19.06212225267312, -49.70194997381457, 0.0)}

#### Chossing 3 images, slicing then in parts of 256x256 and save

In [4]:
path_dir = 'talhoes_tifs'

list_all_images = os.listdir(path_dir)

three_first_images = list_all_images[0:3]

for tif in three_first_images:
    
    results_path = f'Results/{tif}'
    try: 
        os.mkdir(results_path) 
    except OSError as error: 
        print(error) 
        
    utils_tif.crop_image_and_save_windows(f'{path_dir}/{tif}', 256,results_path)

[Errno 17] File exists: 'Results/talhao_ponkan.tif'
[Errno 17] File exists: 'Results/4039_01_04 (1).tif'
[Errno 17] File exists: 'Results/talhao_santa_ana.tif'
